In [ ]:
import torch
from peft import PeftModel, PeftConfig
from transformers import AutoModelForCausalLM, AutoTokenizer

peft_model_id = "./szurkemarha-6.5k"

config = PeftConfig.from_pretrained(peft_model_id)
model = AutoModelForCausalLM.from_pretrained(
    "NYTK/PULI-GPT-3SX", return_dict=True, load_in_8bit=True, device_map="auto"
)

tokenizer = AutoTokenizer.from_pretrained("NYTK/PULI-GPT-3SX")
tokenizer.add_special_tokens({'pad_token': '[PAD]'})

model = PeftModel.from_pretrained(model, peft_model_id, device_map={'':0})

In [ ]:
from transformers import GenerationConfig

generation_config = GenerationConfig(
    temperature=.4,
)
prompt = 'Alább található egy instrukció, ami egy feladatot ír le, valamint egy bemenet, ami további kontextust ad. Írj egy illő választ, ami helyesen teljesíti a kérést!\n\n'
prompt_no_input = 'Alább található egy instrukció, ami egy feladatot ír le. Írj egy illő választ, ami helyesen teljesíti a kérést!\n\n'
data = {'instruction': 'Ajánlj egy sci-fi könyvet!', 'input': None}
#data = {'instruction': 'Add meg a vásárló nevét!', 'input': 'Péter tegnap vett egy autót Petrától.'}
text = (prompt if data['input'] is not None else prompt_no_input)+'### Instrukció:\n' + data['instruction'].strip() + ('\n\n### Bemenet:\n'+data['input'].strip() if data['input'] is not None else '') + '\n\n### Válasz:'
inputs = tokenizer(text, return_tensors="pt")
input_ids = inputs["input_ids"].cuda()
with torch.cuda.amp.autocast():
    output_tokens = model.generate(input_ids=input_ids, do_sample=True, max_new_tokens=64, pad_token_id=tokenizer.pad_token_id, generation_config=generation_config)
    result = tokenizer.decode(output_tokens[0], skip_special_tokens=True)
    print(result)